#### Import Libraries

In [43]:
import pandas as pd
import urllib.request as urllib2
from bs4 import BeautifulSoup
import re
import numpy as np

#### Data URL's

In [2]:
class_def_url = 'http://cogcomp.org/Data/QA/QC/definition.html'
qn_url = 'http://cogcomp.org/Data/QA/QC/train_5500.label'

In [3]:
page1 = urllib2.urlopen(class_def_url)
page2 = urllib2.urlopen(qn_url)

### Page 1

#### Get Data from page

In [4]:
soup = BeautifulSoup(page1, "lxml")

In [5]:
print(soup.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Welcome to Cognitive Computation
  </title>
  <script language="JavaScript">
   <!--
document.write("<LINK REL=\"stylesheet\" HREF=\"");
var ver=parseInt(navigator.appVersion);
var loc="http://l2r.cs.uiuc.edu/~cogcomp/stylesheet/"; 
if((navigator.appName=="Netscape")&&(ver<=4))
{
  document.write(loc+"netscape4.css");
}
else
{
  document.write(loc+"msie.css");
}
document.writeln("\" TYPE=\"text/css\">");

// -->
  </script>
 </head>
 <body>
  <base target="_top"/>
  <center>
   <img height="60" src="/~cogcomp/ccg.gif" width="516"/>
   <br/>
   <h2>
    Definition of Question Classes
   </h2>
  </center>
  <br/>
  <table align="left" border="0" cellpadding="0" cellspacing="0" valign="top" width="100%">
   <tr valign="top">
    <td>
     <blockquote>
      <table border="0" cellpadding="0" cellspacing="0" width="90%">
       <tr align="LEFT" valign="TOP">
        <td width="35%">
         <

#### Generate Table

In [6]:
table = soup.find('tr')

In [7]:
A=[]
B=[]

for row in table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==2: 
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))

In [8]:
class_def_df = pd.DataFrame(A,columns=['Class'])
class_def_df['Definition']=B
class_def_df = class_def_df.loc[1:,:]
class_def_df.head()

,Class,Definition
1,ABBREVIATION,abbreviation
2,&nbsp abb,abbreviation
3,&nbsp exp,expression abbreviated
4,ENTITY,entities
5,&nbsp animal,animals


#### Cleaning Class column in DataFrame

In [9]:
def clean_col(data):
    if '&nbsp' in data:
        return data.split('&nbsp')[1]
    else:
        return data

In [10]:
class_def_df['Class'] = class_def_df['Class'].apply(lambda x: clean_col(x))
class_def_df.head()

,Class,Definition
1,ABBREVIATION,abbreviation
2,abb,abbreviation
3,exp,expression abbreviated
4,ENTITY,entities
5,animal,animals


### Page 2

#### Get Data from page

In [11]:
soup = BeautifulSoup(page2, "lxml")

In [12]:
print(soup.prettify())

<html>
 <body>
  <p>
   DESC:manner How did serfdom develop in and then leave Russia ?
ENTY:cremat What films featured the character Popeye Doyle ?
DESC:manner How can I find a list of celebrities ' real names ?
ENTY:animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?
ABBR:exp What is the full form of .com ?
HUM:ind What contemptible scoundrel stole the cork from my lunch ?
HUM:gr What team did baseball 's St. Louis Browns become ?
HUM:title What is the oldest profession ?
DESC:def What are liver enzymes ?
HUM:ind Name the scar-faced bounty hunter of The Old West .
NUM:date When was Ozzy Osbourne born ?
DESC:reason Why do heavier objects travel downhill faster ?
HUM:ind Who was The Pride of the Yankees ?
HUM:ind Who killed Gandhi ?
ENTY:event What is considered the costliest disaster the insurance industry has ever faced ?
LOC:state What sprawling U.S. state boasts the most airports ?
DESC:desc What did the only repealed amendment to the U.S. Constitution deal w

#### Generate Table

In [13]:
table = soup.find('p')

In [14]:
table

<p>DESC:manner How did serfdom develop in and then leave Russia ?
ENTY:cremat What films featured the character Popeye Doyle ?
DESC:manner How can I find a list of celebrities ' real names ?
ENTY:animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?
ABBR:exp What is the full form of .com ?
HUM:ind What contemptible scoundrel stole the cork from my lunch ?
HUM:gr What team did baseball 's St. Louis Browns become ?
HUM:title What is the oldest profession ?
DESC:def What are liver enzymes ?
HUM:ind Name the scar-faced bounty hunter of The Old West .
NUM:date When was Ozzy Osbourne born ?
DESC:reason Why do heavier objects travel downhill faster ?
HUM:ind Who was The Pride of the Yankees ?
HUM:ind Who killed Gandhi ?
ENTY:event What is considered the costliest disaster the insurance industry has ever faced ?
LOC:state What sprawling U.S. state boasts the most airports ?
DESC:desc What did the only repealed amendment to the U.S. Constitution deal with ?
NUM:count How m

In [15]:
qn_df = pd.DataFrame(str(table)[3:].split('\n'), columns = ['Category1'])
qn_df.head()

,Category1
0,DESC:manner How did serfdom develop in and the...
1,ENTY:cremat What films featured the character ...
2,DESC:manner How can I find a list of celebriti...
3,ENTY:animal What fowl grabs the spotlight afte...
4,ABBR:exp What is the full form of .com ?


#### Create multiple columns in qn_df

In [16]:
qn_df['Questions'] = qn_df['Category1'].apply(lambda x: re.split(' ', x, maxsplit= 1)[1] if ':' in x else x)
qn_df['Category1'] = qn_df['Category1'].apply(lambda x: re.split(' ', x, maxsplit= 1)[0] if ':' in x else x)

In [17]:
qn_df['Category2'] = qn_df['Category1'].apply(lambda x: re.split(':', x, maxsplit= 1)[1] if ':' in x else x)
qn_df['Category1'] = qn_df['Category1'].apply(lambda x: re.split(':', x, maxsplit= 1)[0] if ':' in x else x)

In [18]:
cols = ['Questions', 'Category1', 'Category2']
qn_df = qn_df[cols].drop([5452])

In [19]:
qn_df.head()

,Questions,Category1,Category2
0,How did serfdom develop in and then leave Russ...,DESC,manner
1,What films featured the character Popeye Doyle ?,ENTY,cremat
2,How can I find a list of celebrities ' real na...,DESC,manner
3,What fowl grabs the spotlight after the Chines...,ENTY,animal
4,What is the full form of .com ?,ABBR,exp


In [41]:
short_word = qn_df['Category1'].sort_values().unique()
short_word

array(['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM'], dtype=object)

In [44]:
full_word = np.array(class_def_df[class_def_df['Class'].apply(lambda x: x.isupper())].Class.sort_values())
full_word

array(['ABBREVIATION', 'DESCRIPTION', 'ENTITY', 'HUMAN', 'LOCATION',
       'NUMERIC'], dtype=object)

In [47]:
word_dict = dict(zip(short_word, full_word))

In [53]:
qn_df['Category0'] = qn_df['Category1'].apply(lambda x: word_dict[x])
cols = ['Questions', 'Category0', 'Category1', 'Category2']
qn_df = qn_df[cols]
qn_df.head()

,Questions,Category0,Category1,Category2
0,How did serfdom develop in and then leave Russ...,DESCRIPTION,DESC,manner
1,What films featured the character Popeye Doyle ?,ENTITY,ENTY,cremat
2,How can I find a list of celebrities ' real na...,DESCRIPTION,DESC,manner
3,What fowl grabs the spotlight after the Chines...,ENTITY,ENTY,animal
4,What is the full form of .com ?,ABBREVIATION,ABBR,exp


In [55]:
qn_df.to_csv('Question_Classification_Dataset.csv')